In [ ]:
from flask import Flask, render_template, request
import pickle
import csv
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

app = Flask(__name__)

# Dummy data for movie predictions (replace this with your actual prediction logic)
movie_predictions = []

with open('tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    tfidf_vectorizer = pickle.load(vectorizer_file)
with open('mlb.pkl', 'rb') as mlb_file:
    mlb = pickle.load(mlb_file)

with open('random_forest_classifier.pkl', 'rb') as classifier_file:
    rf_classifier = pickle.load(classifier_file)
    
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    # Get movie name and description from the form
    movie_name = request.form['movie_name']
    movie_description = request.form['movie_description']
    movie_name=f"{movie_name}"
    movie_description=f"{movie_description}"
    
    input_text = [movie_description]
    input_vector = tfidf_vectorizer.transform(input_text).toarray()
    predicted_probabilities = rf_classifier.predict_proba(input_vector)
    
    new=list()
    for i in range(len(predicted_probabilities)):

        new.append(1-predicted_probabilities[i][0][0])
    treshold=.24
    pred=list()
    for i in new:
        if i >= treshold:
            pred.append(1)
        else:
            pred.append(0)
    l=np.array([pred])
    predicted_labels_text = mlb.inverse_transform(l)

    
    # Dummy prediction logic (replace this with your actual prediction logic)
    prediction_label = predicted_labels_text  # Example prediction label
    
    # Store the prediction result
    movie_predictions.append({'name': movie_name, 'description': movie_description, 'prediction': prediction_label})
    with open('Feedback.csv', 'a', newline='') as csvfile:
        fieldnames = ['name', 'description', 'prediction']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({'name': movie_name, 'description': movie_description, 'prediction': prediction_label})
    feedback_data = []
    with open('Feedback.csv', 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            print(row)
            feedback_data.append(row)
    
    
    

    return render_template('result.html', predictions=feedback_data)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Oct/2023 20:20:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 20:23:06] "POST /predict HTTP/1.1" 200 -


{'movie_name': 'Leo', 'description': 'Parthiban is a mild mannered cafe owner in Kashmir, who he fends off a gang of murderous thugs, and gains attention from a drug cartel claiming he was once a part of them.', 'prediction': "[('Action', 'Crime', 'Drama')]"}


127.0.0.1 - - [23/Oct/2023 20:23:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 20:23:56] "POST /predict HTTP/1.1" 200 -


{'movie_name': 'Leo', 'description': 'Parthiban is a mild mannered cafe owner in Kashmir, who he fends off a gang of murderous thugs, and gains attention from a drug cartel claiming he was once a part of them.', 'prediction': "[('Action', 'Crime', 'Drama')]"}
{'movie_name': 'Vikram', 'description': ' A special investigator assigned a case of serial killings discovers the case is not what it seems to be, and leading down this path is only going to end in a war between everyone involved.', 'prediction': "[('Drama', 'Thriller')]"}


127.0.0.1 - - [23/Oct/2023 20:24:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 20:26:47] "POST /predict HTTP/1.1" 200 -


{'movie_name': 'Leo', 'description': 'Parthiban is a mild mannered cafe owner in Kashmir, who he fends off a gang of murderous thugs, and gains attention from a drug cartel claiming he was once a part of them.', 'prediction': "[('Action', 'Crime', 'Drama')]"}
{'movie_name': 'Vikram', 'description': ' A special investigator assigned a case of serial killings discovers the case is not what it seems to be, and leading down this path is only going to end in a war between everyone involved.', 'prediction': "[('Drama', 'Thriller')]"}
{'movie_name': 'Leo', 'description': 'A special investigator assigned a case of serial killings discovers the case is not what it seems to be, and leading down this path is only going to end in a war between everyone involved.', 'prediction': "[('Drama', 'Thriller')]"}


127.0.0.1 - - [23/Oct/2023 20:32:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 20:32:55] "GET /predict HTTP/1.1" 405 -
127.0.0.1 - - [23/Oct/2023 20:33:04] "GET /predict HTTP/1.1" 405 -
127.0.0.1 - - [23/Oct/2023 20:33:11] "GET / HTTP/1.1" 200 -
